### Import python packages

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, Button, HBox, Layout, VBox
from IPython.display import display
import ipywidgets as widgets
from chvisual import CHData
from dataset import Dataset
import IPython
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import networkx as nx
import numpy as np
import ipyspin

### Define the spin for long tasks

In [2]:
s = ipyspin.Spinner()

s.layout.height = '40px'
s.layout.width = '40px'

s.lines = 13  # The number of lines to draw
s.length = 60  # The length of each line
s.width = 8  # The line thickness
s.radius = 8  # The radius of the inner circle
s.scale = 1  # Scales overall size of the spinner
s.corners = 1  # Corner roundness (0..1)
s.speed = 1  # Rounds per second
s.rotate = 0  # The rotation offset
s.animation = 'spinner-line-fade-quick'  # The CSS animation name for the lines
s.direction = 1  # 1: clockwise, -1: counterclockwise
s.color = '#000000'  # CSS color or array of colors
s.fade_color = 'transparent'  # CSS color or array of colors
s.top = '50%'  # Top position relative to parent
s.left = '50%'  # Left position relative to parent
s.shadow = '0 0 1px transparent'  # Box-shadow for the lines
s.z_index = 2000000000  # The z-index (defaults to 2e9)
s.class_name = 'spinner'  # The CSS class to assign to the spinner
s.position = 'absolute'  # Element positioning

s.stop()  # Removes the UI elements from the DOM and stops the animation

s.spin()  # Stopped spinners may be reused by calling spin() again

s  # Displays the spinner

Spinner(class_name='spinner', layout=Layout(height='40px', width='40px'), length=60.0, radius=8.0, width=8.0)

### Create select for the SPARQL endpoint

In [3]:
def set_SPARQL(endpoint):
    print(f'SPARQL is {endpoint}')
    display()
    return endpoint

dataset = Dataset()
     
sparqlWidget = widgets.interactive(set_SPARQL, endpoint=['http://data.bibliotheken.nl/sparql']);

In [4]:
sparqlWidget.result

'http://data.bibliotheken.nl/sparql'

### Init the SPARQL lib and test

In [5]:
chdata = CHData(sparqlWidget.result, 30)

### Retrieve authors from the selected SPARQL endpoint and create the interface

In [6]:
def set_Author(author):
    out = widgets.Output()
    display(out)
    
    with out:
        display(s)

        idAuthor = author[author.rindex('-') +2:]
        authorData = chdata.getAuthorInfo(idAuthor)
        metadataItems = []
        optionsItems = []
     
        p = None
        metadata = ''
        for r in authorData:
            propertyLink = "<a href=" + r['p']['value'] +  "><font color='blue'>"+ r['p']['value'] + "</font></a>"
            metadata = metadata + "<p>" + propertyLink + ": " + r['o']['value'] + "</p>" 
        p = widgets.HTML(metadata)
        metadataItems.append(p)
    
        #img = None
        #url = chdata.getDepiction(authorData)
        #if url != None and url != '':
        #    img = widgets.HTML("<img width='50%' src='"+url+"'>")
        #    optionsItems.append(img)
    
        downloadButton = widgets.Button(
            description='Download metadata',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Download metadata',
            icon='download' # (FontAwesome names without the `fa-` prefix)
        )
    
        def on_button_clicked(b):
            chdata.downloadGraph(idAuthor)
            
        downloadButton.on_click(on_button_clicked)
        optionsItems.append(downloadButton)
    
        optionsLayout = VBox()
        optionsLayout.children = optionsItems
        metadataItems.append(optionsLayout)
    
        metadataLayout = HBox()
        metadataLayout.children = metadataItems
    
        widget_plot_graph = widgets.Output(layout=Layout(width='70%'))
        with widget_plot_graph:
            IPython.display.clear_output(True)
            g = chdata.getGraph(idAuthor.replace("/doc/", "/id/"))
    
            colors = []
            for u in g.nodes(data="nodetype"):
                if u[1] in ("red", "green", "blue","yellow", "cyan", "magenta"):
                    colors.append(u[1])
                elif u[1] == None:
                    colors.append("blue")
                else:
                    #do something?
                    print("ERROR: Should be red, green, blue or None")
    
            legend_elements = [
                Line2D([0], [0], marker='o', color='w', label='Author class',markerfacecolor='g', markersize=15),
                Line2D([0], [0], marker='o', color='w', label='Work class',markerfacecolor='r', markersize=15),
                Line2D([0], [0], marker='o', color='w', label='Author node',markerfacecolor='y', markersize=15),
                Line2D([0], [0], marker='o', color='w', label='Other',markerfacecolor='b', markersize=15),
                Line2D([0], [0], marker='o', color='w', label='Manifestation class',markerfacecolor='m', markersize=15),
                Line2D([0], [0], marker='o', color='w', label='Concept class',markerfacecolor='c', markersize=15),
            ]
            
            nx.draw_networkx(g, node_color=colors, with_labels=False)
            plt.legend(handles=legend_elements, loc='upper right')
            plt.show()
    
        worksData = chdata.getAuthorWorks(idAuthor)
        worksWidget = None
        worksText = ''
        for r in worksData:
            workLink = "<a href=" + r['s']['value'] +  "><font color='blue'>"+ r['s']['value'] + "</font></a>"
            worksText = worksText + "<p>" + workLink + ": " + r['label']['value'] + "</p>" 
        worksWidget = widgets.HTML(worksText)
    
        accordionChildren = []
        accordionChildren.append(metadataLayout)
        accordionChildren.append(worksWidget)
        accordionChildren.append(widget_plot_graph)
        authorLayout = widgets.Accordion(children=accordionChildren, titles=('Metadata', 'Works', 'Graph'))
        out.clear_output()
        display(authorLayout)
    
    return author

authorWidget = widgets.interactive(set_Author, author=chdata.getAuthors());

### Retrieve classes from the repository

In [7]:
classesItems = []
p = None
classesText = ''
classesData = chdata.getClasses()
for i in classesData[0]:
    classLink = "<a href=" + i +  "><font color='blue'>"+ i + "</font></a>"
    classesText = classesText + "<p>" + classLink + "</p>" 
p = widgets.HTML(classesText)
classesItems.append(p)

widget_plot_classes = widgets.Output(layout=Layout(width='50%'))
with widget_plot_classes:
    IPython.display.clear_output(True)
    X = np.array(list(classesData[1].keys()))
    Y = np.array(list(classesData[1].values()))
    plt.xticks(rotation='vertical')
    fig = plt.bar(X, Y, color ='blue', width = 0.6)
    plt.show()
classesItems.append(widget_plot_classes)

classesLayout = HBox()
classesLayout.children = classesItems

### Retrieve properties from the repository

In [8]:
propertiesItems = []
p = None
propertiesText = ''
propertiesData = chdata.getProperties()

for i in propertiesData[0]:
    propertyLink = "<a href=" + i +  "><font color='blue'>"+ i + "</font></a>"
    propertiesText = propertiesText + "<p>" + propertyLink + "</p>" 
p = widgets.HTML(propertiesText)
propertiesItems.append(p)

widget_plot_properties = widgets.Output(layout=Layout(width='50%'))
with widget_plot_properties:
    IPython.display.clear_output(True)
    X = np.array(list(propertiesData[1].keys()))
    Y = np.array(list(propertiesData[1].values()))
    plt.xticks(rotation='vertical')
    fig = plt.bar(X, Y, color ='blue', width = 0.6)
    plt.show()
propertiesItems.append(widget_plot_properties)

propertiesLayout = HBox()
propertiesLayout.children = propertiesItems

### Create Wikidata visualisation

In [9]:
wikidata_file = open('map/nl-wikidata-map.html', "r")
wikidataWidget = widgets.HTML(wikidata_file.read())

### Create visual interface

In [10]:
children = []
titles = []
tab = widgets.Tab()
children.insert(0,sparqlWidget)
children.insert(1,authorWidget)
children.insert(2,classesLayout)
children.insert(3,propertiesLayout)
children.insert(4,wikidataWidget)
tab.children = children

titles.insert(0, 'SPARQL endpoint')
titles.insert(1, 'Author')
titles.insert(2, 'Classes')
titles.insert(3, 'Properties')
titles.insert(4, 'Wikidata')
tab.titles = titles

tab